In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*6)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [5]:
base_dir = '../data/telemoji/pre'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

train_datagen = ImageDataGenerator(
        rescale=1/255,    
#        rotation_range=40,
#        width_shift_range=0.1,
#        height_shift_range=0.1,
#        zoom_range=0.2,
#        horizontal_flip=True,
#        vertical_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
     rescale=1/255
)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    color_mode='grayscale',
    target_size=(48,48),
    batch_size=20,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    color_mode='grayscale',
    target_size=(150,150),
    batch_size=20,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    validation_dir,
    color_mode='grayscale',
    target_size=(150,150),
    batch_size=20,
    class_mode='categorical'
)

Found 15986 images belonging to 7 classes.
Found 3992 images belonging to 7 classes.
Found 2074 images belonging to 7 classes.


# tensroflow 2

In [21]:
for data_batch, lable_batch in train_generator:
    ### block 1 ###
    C1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(data_batch)
    C2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(C1)

    P1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(C2)
    P1 = tf.keras.activations.relu(P1)

    ### block 2 ###
    C3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(P1)
    C4 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(C3)
    C5 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(C4)

    P2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(C5)
    P2 = tf.keras.activations.relu(P2)

    FC1 = tf.keras.layers.Flatten()(P2)
    FC1 = tf.keras.layers.Dense(256)(FC1)

    ### block 3 ###
    C6 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(P2)
    C7 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C6)
    C8 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C7)
    C9 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C8)

    P3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(C9)
    P3 = tf.keras.activations.relu(P3)

    FC2 = tf.keras.layers.Flatten()(P3)
    FC2 = tf.keras.layers.Dense(256)(FC2)

    ### block 4 ###
    C10 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(P3)
    C11 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C10)
    C12 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C11)
    C13 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C12)

    P4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(C13)
    P4 = tf.keras.activations.relu(P4)

    FC3 = tf.keras.layers.Flatten()(P4)
    FC3 = tf.keras.layers.Dense(256)(FC3)

    ### block 5 ###
    C14 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same')(P4)
    C15 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same')(C14)
    C16 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same')(C15)
    C17 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same')(C16)

    P5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(C17)
    P5 = tf.keras.activations.relu(P5)
    
    FC4 = tf.keras.layers.Flatten()(P5)

    FC5 = tf.concat([FC1, FC2, FC3, FC4], 1)
    FC5 = tf.keras.layers.Dense(7, activation='softmax')(FC4)

    ### loss function ###
    loss = tf.keras.losses.categorical_crossentropy(lable_batch, FC5)

    ### train ###
    opt = tf.keras.optimizers.RMSprop(learning_rate=1e-4)
    train = opt.minimize(loss, [FC5])
    break
print(train)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object is not callable

In [ ]:
### block 1 ###
C1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')()
C2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(C1)

P1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(C2)
P1 = tf.keras.activations.relu(P1)

### block 2 ###
C3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(P1)
C4 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(C3)
C5 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(C4)

P2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(C5)
P2 = tf.keras.activations.relu(P2)

FC1 = tf.keras.layers.Flatten()(P2)
FC1 = tf.keras.layers.Dense(256)(FC1)

### block 3 ###
C6 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(P2)
C7 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C6)
C8 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C7)
C9 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C8)

P3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(C9)
P3 = tf.keras.activations.relu(P3)

FC2 = tf.keras.layers.Flatten()(P3)
FC2 = tf.keras.layers.Dense(256)(FC2)

### block 4 ###
C10 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(P3)
C11 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C10)
C12 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C11)
C13 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(C12)

P4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(C13)
P4 = tf.keras.activations.relu(P4)

FC3 = tf.keras.layers.Flatten()(P4)
FC3 = tf.keras.layers.Dense(256)(FC3)

### block 5 ###
C14 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same')(P4)
C15 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same')(C14)
C16 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same')(C15)
C17 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, activation='relu', padding='same')(C16)

P5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(C17)
P5 = tf.keras.activations.relu(P5)

FC4 = tf.concat([FC1, FC2, FC3, P5])
FC4 = tf.keras.layers.Dense(7, activation='softmax')(FC4)

### loss function ###
loss = tf.keras.losses.categorical_crossentropy(y_true, FC4)

### train ###
opt = tf.keras.optimizers.RMSprop(learning_rate=1e-4)
train = opt.minimize(loss, [FC4])